<a href="https://colab.research.google.com/github/mgite03/image-preprocessing/blob/main/github_text_recognition_in_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic cv2 functionality

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow

# or whatever filepath the image has, this was done on google colab
filepath = '/content/img_1.jpg'

#NOT GRAYSCALE
#img = cv2.imread(filepath)
#GRAYSCALE
img = cv2.imread(filepath, 0)

cv2_imshow(img)

#ALTERNATE WAY OF SHOWING IMAGES, INSTEAD OF AS AN IMAGE ITS PYPLOT - doesn't really work w cv grayscale
#plt.imshow(img)

#imwrite writes to a file
cv2.imwrite('MyPicGray.jpg', img)

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
import pytesseract

import difflib


In [ ]:
#takes in and image file and its corresponding text file, and outputs the ratio of how similar they are
def read_image_and_compare_truth(img, textfile,flag):
  text = ""

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # do something called thresholding - under the first number everything turns a specific color - cv2.THRESH_BINARY_INV and cv2.THRESH_OTSU were the two flags tested
  ret, thresh1 = cv2.threshold(gray, 120, 255, cv2.THRESH_OTSU)
  
  #plt.imshow(thresh1)

  # Specify structure shape and kernel size. 
  # Kernel size increases or decreases the area 
  # of the rectangle to be detected.
  # A smaller value like (10, 10) will detect 
  # each word instead of a sentence
  rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
  kernel = np.ones((20,20),np.float32)/400


    # dst = cv2.filter2D(img,-1,kernel)
    # blur = cv2.blur(img,(20,20))



  #dilate the image
  if flag == 1:
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
  else:
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 3)
    
  #finding contours  
  contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, 
                                                  cv2.CHAIN_APPROX_NONE)

  # Creating a copy of image
  im2 = img.copy()
    

    
  # Looping through the identified contours
  # Then rectangular part is cropped and passed on
  # to pytesseract for extracting text from it
  # Extracted text is then written into the text file
  for cnt in contours:
      x, y, w, h = cv2.boundingRect(cnt)
        
      # Drawing a rectangle on copied image
      rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 255), 2)
        
      # Cropping the text block for giving input to OCR
      cropped = im2[y:y + h, x:x + w]
        
      # Open the file in append mode
      #file = open("recognized.txt", "a")
        
      # Apply OCR on the cropped image
      text += pytesseract.image_to_string(cropped)




  gt = textfile
  filelines = gt.readlines()
  ret_str = ""
  for line in filelines:                
      splitline = line.split()
      words = splitline[4]

      ret_str += words


  ratio = difflib.SequenceMatcher(None, ret_str,text).ratio()
  return ratio

img = cv2.imread("/content/100.jpg")
textfile = open("/content/gt_100.txt", mode="r")

#flag was changed depending on whether thesholding, dilation, or 
flag = 0
read_image_and_compare_truth(img, textfile,flag)

In [ ]:
#testing differences in dilation, can be changed for blur and thresholding
similarity = []
imgs = []
ones = []
threes = []
for i in range (100,131):
  imgs += [i]

  tex = open(f"/content/gt_{i}.txt", mode = 'r')
  imag = cv2.imread(f"/content/{i}.jpg")
  one = read_image_and_compare_truth(imag, tex, 1)
  ones += [one]

  tex = open(f"/content/gt_{i}.txt", mode = 'r')
  imag = cv2.imread(f"/content/{i}.jpg")
  three = read_image_and_compare_truth(imag, tex, 3)
  threes += [threes]

  similarity += [three-one]



In [ ]:
plt.plot(imgs, similarity, "co")


# plt.hist(similarity, 10)
# plt.xlim(0,1)
plt.axhline(y=0, color="black", linestyle="-")
plt.title('Dilation Differences(3 Iterations - 1 Iteration)')
plt.xlabel('Picture number')
plt.ylabel('Fraction of Matching Characters')

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(otsu, 10, label="OTSU")

plt.xlabel('Pic Number')
plt.ylabel('Fraction of Matching Characters')
plt.title('Thresholding Differences')

plt.legend(loc="upper left")

plt.subplot(1, 2, 2)

plt.hist(binary, 10, label="BINARY_INV")
plt.xlabel('Pic Number')
plt.ylabel('Fraction of Matching Characters')
plt.title('Thresholding Differences')

plt.legend(loc="upper left")
plt.show()

# easy ocr

In [ ]:
!pip install easyocr

import easyocr
reader = easyocr.Reader(['en']) # need to run only once to load model into memory

In [ ]:
result = reader.readtext("/content/167.jpg", detail=0, paragraph= True)
print(result)

img = cv2.imread("/content/167.jpg")
cv2_imshow(img)
